In [26]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import praw
from textblob import TextBlob
from sqlalchemy import create_engine, text

In [8]:
# loading the env variables
load_dotenv()

True

In [9]:
client_secret = os.getenv("APP_SECRET_KEY")
client_id = os.getenv("APP_CLIENT_ID")
user_agent_url = os.getenv("APP_USER_AGENT")
database_url = os.getenv("DATABASE_URL")

In [10]:
# creating connection and engine
engine = create_engine(database_url)

In [11]:
# ---- checking to make sure everything is here -----
if not all([client_secret, client_id, user_agent_url]):
    raise ValueError("Missing one or more required environment variables: APP_SECRET_KEY, APP_CLIENT_ID, APP_USER_AGENT")

## Setting up reddit

In [12]:
reddit = praw.Reddit(
    client_id= client_id,
    client_secret= client_secret,
    user_agent=user_agent_url)

In [13]:
# check to make sure we are using the right version
reddit.read_only

True

## Extrating posts from a subreddit

In [14]:
someSubredditName = "csMajors"
subreddit = reddit.subreddit(someSubredditName)
top_posts = subreddit.top(limit=50, time_filter="day")

In [15]:
raw_data = []
for post in top_posts:
    raw_data.append({
        "post_id": post.id,
        "title": post.title,
        "author": str(post.author),
        "score": post.score,
        "num_comments": post.num_comments,
        "created_utc": post.created_utc
    })

In [16]:
len(raw_data)

50

In [17]:
raw_data[0]

{'post_id': '1jdvbiy',
 'title': 'Microsoft commented on my TikTok, how can I turn this into an interview?',
 'author': 'AdWooden391',
 'score': 945,
 'num_comments': 35,
 'created_utc': 1742266410.0}

## Transform Phse
1. Data Cleaning: Remove null authors, standardize timestamps, handle text encoding
2. Sentiment Analysis: Using textblob to get sentiment polarity
3. Data MOdeling: convert timestamps to a standard datetime

In [18]:
df = pd.DataFrame(raw_data)

In [19]:
df.head()

,post_id,title,author,score,num_comments,created_utc
0,1jdvbiy,"Microsoft commented on my TikTok, how can I tu...",AdWooden391,945,35,1.742266e+09
1,1je61q2,Imagine picking a major so prone to automation...,RegardedEpicGamer,456,149,1.742308e+09
2,1jdtjba,How it feels to get an internship,davms87,140,4,1.742261e+09
3,1jdmfki,Should I take an AI internship from Huawei? Wo...,mhadv102,117,66,1.742242e+09
4,1jdu62m,Pity the 2025 dec grads and mid 2026 grads.,Dramatic-Fall701,64,68,1.742263e+09


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   post_id       50 non-null     object 
 1   title         50 non-null     object 
 2   author        50 non-null     object 
 3   score         50 non-null     int64  
 4   num_comments  50 non-null     int64  
 5   created_utc   50 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 2.5+ KB


In [21]:
df["created_utc"] = pd.to_datetime(df["created_utc"])

In [22]:
df.head(n=1)

,post_id,title,author,score,num_comments,created_utc
0,1jdvbiy,"Microsoft commented on my TikTok, how can I tu...",AdWooden391,945,35,2025-03-18 02:53:30


In [23]:
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"
    
df["sentiment"] = df["title"].apply(get_sentiment)

In [24]:
df.head(n=1)

,post_id,title,author,score,num_comments,created_utc,sentiment
0,1jdvbiy,"Microsoft commented on my TikTok, how can I tu...",AdWooden391,945,35,2025-03-18 02:53:30,Neutral


## Loading Phase

In [25]:
unique_authors = df["author"].unique()

In [44]:
with engine.begin() as conn:
    for author in unique_authors:
        # Try to get existing author id
        author_id = conn.execute(
            text("SELECT id FROM dim_authors WHERE author_name = :author"),
            {"author": author}
        ).scalar()
        # If not found, insert new row and get id
        if author_id is None:
            author_id = conn.execute(
                text("INSERT INTO dim_authors (author_name) VALUES (:author) RETURNING id"),
                {"author": author}
            ).scalar()
        author_mapping[author] = author_id

# ----- Load dim_subreddit -----
print("Loading dim_subreddit...")
# Assuming the project is for a constant subreddit (e.g., "AskScience")
subreddit_name = someSubredditName 
with engine.begin() as conn:
    subreddit_id = conn.execute(
        text("SELECT id FROM dim_subreddit WHERE subreddit_name = :subreddit"),
        {"subreddit": subreddit_name}
    ).scalar()
    if subreddit_id is None:
        subreddit_id = conn.execute(
            text("INSERT INTO dim_subreddit (subreddit_name) VALUES (:subreddit) RETURNING id"),
            {"subreddit": subreddit_name}
        ).scalar()

# ----- Load dim_date -----
print("Loading dim_date...")
unique_dates = df['created_utc'].dt.date.unique()
date_mapping = {}

with engine.begin() as conn:
    for d in unique_dates:
        date_id = conn.execute(
            text("SELECT date_id FROM dim_date WHERE date_value = :date_value"),
            {"date_value": d}
        ).scalar()
        if date_id is None:
            year = d.year
            month = d.month
            day_of_week = d.weekday()  # Monday = 0
            date_id = conn.execute(
                text("""
                    INSERT INTO dim_date (date_value, year, month, day_of_week) 
                    VALUES (:date_value, :year, :month, :day_of_week) 
                    RETURNING date_id
                """),
                {"date_value": d, "year": year, "month": month, "day_of_week": day_of_week}
            ).scalar()
        date_mapping[str(d)] = date_id

# ----- Load fact_posts -----
print("Loading fact_posts...")
print("Loading fact_posts and dim_posts...")
with engine.begin() as conn:
    for _, row in df.iterrows():
        author_id = author_mapping[row['author']]
        date_str = str(row['created_utc'].date())
        date_id = date_mapping[date_str]
        
        # Insert into fact_posts and capture the fact_id using RETURNING
        result = conn.execute(
            text("""
                INSERT INTO fact_posts 
                (post_id, author_id, subreddit_id, score, num_comments, sentiment, date_id)
                VALUES 
                (:post_id, :author_id, :subreddit_id, :score, :num_comments, :sentiment, :date_id)
                RETURNING fact_id
            """),
            {
                "post_id": row['post_id'],
                "author_id": author_id,
                "subreddit_id": subreddit_id,
                "score": row['score'],
                "num_comments": row['num_comments'],
                "sentiment": row['sentiment'],
                "date_id": date_id
            }
        )
        fact_id = result.scalar()
        
        # Insert into dim_posts with the returned fact_id and the title
        conn.execute(
            text("""
                INSERT INTO dim_posts (author_id, fact_post_id, title)
                VALUES (:author_id, :fact_post_id, :title)
            """),
            {
                "author_id": author_id,
                "fact_post_id": fact_id,
                "title": row['title']
            }
        )
print("Loading Done")

Loading dim_subreddit...
Loading dim_date...
Loading fact_posts...
Loading fact_posts and dim_posts...
Loading Done
